# Converting unstructured test set

In [ ]:
import pandas as pd
import os, pathlib, shutil

# whole data directory
datapath = pathlib.Path("/home/mpuscian/Desktop/repozytoria/MINI_projects/Transformers/data/")

# source path with all train, validation and test files from kaggle
source_path = datapath.joinpath("raw/train/")

destination_path = datapath.joinpath("processed/splited_data/")

testing_list_df = pd.read_csv(datapath.joinpath("raw/testing_list.txt"), names=["filename"])
validation_list_df = pd.read_csv(datapath.joinpath("raw/validation_list.txt"), names=["filename"])

testing_list = testing_list_df.filename.to_list()
validation_list = validation_list_df.filename.to_list()
print(f"No. testing files: {len(testing_list)}\nNo. validation files: {len(validation_list)}")


No. testing files: 6835
No. validation files: 6798


In [ ]:
def get_ignore_func(files_to_ignore):
    def ignore_func(directory, contents):
        # Get paths relative to the root directory
        ignore_set = set()
        for path in files_to_ignore:
            # Extract just the filename from the path, assuming the format 'folder/filename'
            _, filename = os.path.split(path)
            ignore_set.add(filename)
        
        # Return files that should be ignored in the current directory
        return [item for item in contents if item in ignore_set]
    
    return ignore_func

def copy_specific_files(source_root, dest_root, file_list):
    # Ensure destination directory exists
    os.makedirs(dest_root, exist_ok=True)
    
    # Process each file in the list
    for file_path in file_list:
        folder, filename = os.path.split(file_path)
        
        # Create the destination folder if it doesn't exist
        dest_folder = os.path.join(dest_root, folder)
        os.makedirs(dest_folder, exist_ok=True)
        
        # Copy the file
        source_file = os.path.join(source_root, file_path)
        dest_file = os.path.join(dest_root, file_path)
        
        if os.path.exists(source_file):
            shutil.copy2(source_file, dest_file)
        else:
            print(f"Warning: Source file not found: {source_file}")

In [ ]:
# create folders if don't exist
destination_train = pathlib.Path(destination_path).joinpath("train")
destination_validation = pathlib.Path(destination_path).joinpath("validation")
destination_test = pathlib.Path(destination_path).joinpath("test")

destination_train.mkdir(parents=True, exist_ok=True)
destination_validation.mkdir(parents=True, exist_ok=True)
destination_test.mkdir(parents=True, exist_ok=True)

all_exclude_files = testing_list + validation_list

shutil.copytree(source_path, destination_train.as_posix(), ignore=get_ignore_func(all_exclude_files), dirs_exist_ok=True)
copy_specific_files(source_path, destination_validation, validation_list)
copy_specific_files(source_path, destination_test, testing_list)
